In [ ]:
import pathlib as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import simple_soil

Conversion factors

In [ ]:
in2m = 1.0 / (12.0 * 3.28081)
acre2sqm = 4046.86

hru_1 parameters

In [ ]:
area = 2715.617356 * acre2sqm
factor = 10000.0
max_vertical_rate = 0.05  # 0.08239 * in2m * factor
horizontal_vertical_ratio = 1.0  # 0.01 * in2m * factor / max_vertical_rate

In [ ]:
theta_sat = 0.2
theta_fc = 0.1
theta_wp = 0.01

In [ ]:
thickness = 3.0  # 0.659252 * in2m / theta_fc
thickness

Forcing data

In [ ]:
tag = " 2"

In [ ]:
# rainfall
rain = pd.read_csv(pl.Path("data/hru_1_rain.csv"), index_col=0)
rain["ppt m/day"] = rain[tag] * in2m
rain = rain.drop(tag, axis=1)
rain.columns

In [ ]:
# canopy et
canopy_et = pd.read_csv(pl.Path("data/hru_1_intcpevap.csv"), index_col=0)

In [ ]:
# potential
pet = pd.read_csv(pl.Path("data/hru_1_potet.csv"), index_col=0)

In [ ]:
ax = canopy_et.plot(label="canopy")
pet.plot(ax=ax, label="pet")

In [ ]:
# soil pet
pet["pet m/day"] = (pet[tag] - canopy_et[tag]) * in2m
pet = pet.drop(tag, axis=1)
pet.columns

In [ ]:
ax = rain.plot()
pet.plot(ax=ax, label="pet")

Control volume

In [ ]:
cv = simple_soil.base.ControlVolume(
    area=area,
    thickness=thickness,
    max_vertical_rate=max_vertical_rate,
    horizontal_vertical_ratio=horizontal_vertical_ratio,
    theta0=0.1,
    theta_wp=theta_wp,
    theta_fc=theta_fc,
    theta_sat=theta_sat,
)

In [ ]:
cv

In [ ]:
for inflow, pet_rate in zip(rain["ppt m/day"][0:366], pet["pet m/day"][0:366]):
    cv.update(inflow_rate=inflow, pet_rate=pet_rate * 0.5)

In [ ]:
df = cv.get_dataframe(normalize=True)

In [ ]:
df

In [ ]:
ax = (df["rejected_inflow_L/T"] + df["inflow_L/T"]).plot()
df["rejected_inflow_L/T"].plot(ax=ax)
df["inflow_L/T"].plot(ax=ax)
ax.legend()

In [ ]:
ax = df["theta"].plot()
ax.axhline(cv.theta_fc, lw=0.5, color="0.5")
ax.axhline(cv.theta_discharge, lw=0.5, color="0.5")
ax.axhline(cv.theta_sat, lw=0.5, color="0.5")

In [ ]:
ax = df["recharge_L/T"].plot()
df["lateral_L/T"].plot(ax=ax, ls="--")
df["storage_change_L/T"].plot(ax=ax)
ax.legend()